In [ ]:
import numpy as np
from itertools import islice
from scipy.stats import multivariate_normal

def performance(y, pred):
  m = np.zeros((3,3)) # confusion matrix
  for p in range(len(pred)):
    if pred[p]==1 and y[p]==1:
      m[0,0]+=1
    if pred[p]==2 and y[p]==2:
      m[1,1]+=1
    if pred[p]==3 and y[p]==3:
      m[2,2]+=1
    if pred[p]==1 and y[p]==2:
      m[1,0]+=1
    if pred[p]==1 and y[p]==3:
      m[2,0]+=1
    if pred[p]==2 and y[p]==1:
      m[0,1]+=1
    if pred[p]==2 and y[p]==3:
      m[2,1]+=1
    if pred[p]==3 and y[p]==1:
      m[0,2]+=1
    if pred[p]==3 and y[p]==2:
      m[1,2]+=1
  ind_accuracy = [m[0,0]/np.sum(m[0,:]), m[1,1]/np.sum(m[1,:]), m[2,2]/np.sum(m[2,:])]
  accuracy = (m[0,0]+m[1,1]+m[2,2])/np.sum(m)
  return ind_accuracy, accuracy

In [ ]:
# getting data from file
data = []
filename = '/content/drive/MyDrive/NNFL Assignments (Aug 2021)/Assignment 1/data_q6_q7.txt'
with open(filename) as file:
  lines = list(islice(file,None,None,None))
  for line in lines:
    entries = line.split('\t')
    while '' in entries: entries.remove('')
    y = int(entries[-1].replace('\n',''))
    entries.pop()
    v = list(map(float,entries))
    v.append(y)
    data.append(v)

data = np.concatenate((np.ones((np.shape(data)[0],1)),np.array(data)), axis=1) #appending ones
np.random.shuffle(data) # shuffling data
x = np.array(data[:,:np.shape(data)[1]-1])
y = np.array(data[:,-1])

In [ ]:
m = len(y)
nf = 5 #number of folds
x_subsets = np.array_split(x, nf)
y_subsets = np.array_split(y, nf)

In [ ]:
# MAP Classifier
accuracy_vals = []
ind_accuracy1 = []
ind_accuracy2 = []
ind_accuracy3 = []

for fold in range(nf):
  # test-train split
  x_test = x_subsets[fold]
  y_test = y_subsets[fold]

  x_train = np.concatenate(np.delete(x_subsets, fold, 0), axis=0)
  y_train = np.concatenate(np.delete(y_subsets, fold, 0), axis=0)

  m_train = len(y_train)
  m_test = len(y_test)

  # normalizing input data
  pp = np.amax(np.abs(x_train), axis=0)
  x_train = x_train/pp
  x_test = x_test/pp

  # data for likelihood
  x_class1_train = x_train[np.where(y_train==1)]
  x_class2_train = x_train[np.where(y_train==2)]
  x_class3_train = x_train[np.where(y_train==3)]

  y_class1_train = y_train[np.where(y_train==1)]
  y_class2_train = y_train[np.where(y_train==2)]
  y_class3_train = y_train[np.where(y_train==3)]

  # likelihood modeled by multivariate gaussian
  # pdfXgiven1 = multivariate_normal.pdf(x_test, np.mean(x_class1_train, axis=0), np.cov(x_class1_train, rowvar=False), allow_singular=True)
  # pdfXgiven2 = multivariate_normal.pdf(x_test, np.mean(x_class2_train, axis=0), np.cov(x_class1_train, rowvar=False), allow_singular=True)
  # pdfXgiven3 = multivariate_normal.pdf(x_test, np.mean(x_class3_train, axis=0), np.cov(x_class3_train, rowvar=False), allow_singular=True)

  mean1 = np.mean(x_class1_train, axis=0)
  mean2 = np.mean(x_class2_train, axis=0)
  mean3 = np.mean(x_class3_train, axis=0)

  cov1 =np.cov(x_class1_train, rowvar=False)
  cov2 =np.cov(x_class1_train, rowvar=False)
  cov2 =np.cov(x_class3_train, rowvar=False)

  pdfXgiven1 = np.exp(-1*x_test/mean1)/mean1
  pdfXgiven2 = np.exp(-1*x_test/mean2)/mean2
  pdfXgiven3 = np.exp(-1*x_test/mean3)/mean3

  pofy1 = len(y_class1_train)/m_train
  pofy2 = len(y_class2_train)/m_train
  pofy3 = len(y_class3_train)/m_train

  # not considering evidence as it is constant for all classes

  MAP_class1 = pdfXgiven1 * pofy1
  MAP_class2 = pdfXgiven2 * pofy2
  MAP_class3 = pdfXgiven3 * pofy3

  # prediction
  pred = []
  for i in range(m_test):
    MAP_vector = [MAP_class1[i],MAP_class2[i],MAP_class3[i]]
    pred.append(np.argmax(MAP_vector)+1)

  # performance measures
  ind_accuracy, accuracy = performance(y_test, pred)
  accuracy_vals.append(accuracy)
  ind_accuracy1.append(ind_accuracy[0])
  ind_accuracy2.append(ind_accuracy[1])
  ind_accuracy3.append(ind_accuracy[2])

mean_accuracy = np.mean(accuracy_vals)
mean_accuracy1 = np.mean(ind_accuracy1)
mean_accuracy2 = np.mean(ind_accuracy2)
mean_accuracy3 = np.mean(ind_accuracy3)

print("mean accuracy of class 1 = {}".format(mean_accuracy1))
print("mean accuracy of class 2 = {}".format(mean_accuracy3))
print("mean accuracy of class 3 = {}".format(mean_accuracy3))
print("mean accuracy of classifier = {}".format(mean_accuracy))

mean accuracy of class 1 = 0.9125506072874494
mean accuracy of class 2 = 0.971291866028708
mean accuracy of class 3 = 0.971291866028708
mean accuracy of classifier = 0.9523809523809523
